# News dataset loading

In [1]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [2]:
path = "../data/news/nasdaq_external_data.csv"

news_df = pd.read_csv(path)

C:\Users\crmgr\AppData\Local\Temp\ipykernel_9372\1676302587.py:3: DtypeWarning: Columns (3,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  news_df = pd.read_csv(path)


# News dataset inpection

In [31]:
news_df.head()

,Unnamed: 0,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,0.0,2023-12-16 23:00:00 UTC,Interesting A Put And Call Options For August ...,A,https://www.nasdaq.com/articles/interesting-a-...,NaN,NaN,"Investors in Agilent Technologies, Inc. (Symbo...",Because the $125.00 strike represents an appro...,The current analytical data (including greeks ...,Below is a chart showing the trailing twelve m...,"At Stock Options Channel, our YieldBoost formu..."
1,1.0,2023-12-12 00:00:00 UTC,Wolfe Research Initiates Coverage of Agilent T...,A,https://www.nasdaq.com/articles/wolfe-research...,NaN,NaN,"Fintel reports that on December 13, 2023, Wolf...","Fintel reports that on December 13, 2023, Wolf...","T. Rowe Price Investment Management holds 10,1...",Agilent Technologies Declares $0.24 Dividend O...,The projected annual revenue for Agilent Techn...
2,2.0,2023-12-12 00:00:00 UTC,Agilent Technologies Reaches Analyst Target Price,A,https://www.nasdaq.com/articles/agilent-techno...,NaN,NaN,"In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...",When a stock reaches the target an analyst has...,When a stock reaches the target an analyst has...
3,3.0,2023-12-07 00:00:00 UTC,Agilent (A) Enhances BioTek Cytation C10 With ...,A,https://www.nasdaq.com/articles/agilent-a-enha...,NaN,NaN,Agilent Technologies A is enhancing its BioTek...,"Per a Grand View Research report, the global m...","Notably, Agilent enhanced the BioTek Cytation ...","Agilent Technologies, Inc. Price and Consensus...","Notably, Agilent enhanced the BioTek Cytation ..."
4,4.0,2023-12-07 00:00:00 UTC,"Pre-Market Most Active for Dec 7, 2023 : SQQQ,...",A,https://www.nasdaq.com/articles/pre-market-mos...,NaN,NaN,The NASDAQ 100 Pre-Market Indicator is up 70.2...,ProShares UltraPro Short QQQ (SQQQ) is -0.15 a...,"As reported by Zacks, the current mean recomme...","The total Pre-Market volume is currently 39,23...",The NASDAQ 100 Pre-Market Indicator is up 70.2...


In [32]:
print(len(news_df))
print(news_df.columns)

15549299
Index(['Unnamed: 0', 'Date', 'Article_title', 'Stock_symbol', 'Url',
       'Publisher', 'Author', 'Article', 'Lsa_summary', 'Luhn_summary',
       'Textrank_summary', 'Lexrank_summary'],
      dtype='object')


In [33]:
# Show the unique values of every column and their counts
for col in news_df.columns:
    print(f"Unique values in {col}: {news_df[col].nunique()}")
    print(news_df[col].unique())
    print()

Unique values in Unnamed: 0: 2491785
[0.000000e+00 1.000000e+00 2.000000e+00 ... 2.491783e+06 2.491784e+06
          nan]

Unique values in Date: 2796596
['2023-12-16 23:00:00 UTC' '2023-12-12 00:00:00 UTC'
 '2023-12-07 00:00:00 UTC' ... '2013-11-10 02:40:00 UTC'
 '2013-11-10 02:39:00 UTC' '2013-11-10 00:46:00 UTC']

Unique values in Article_title: 10452636
['Interesting A Put And Call Options For August 2024'
 'Wolfe Research Initiates Coverage of Agilent Technologies (A) with Outperform Recommendation'
 'Agilent Technologies Reaches Analyst Target Price' ...
 'Apple Said to Add Foxconn’s Chimei as Supplier of Parts for IPad'
 'CEZ, Bank Pekao Shares May Move: Central European Stock Preview'
 'Popolare di Milano May Join Intesa, Paschi in Selling Shares This Year']

Unique values in Stock_symbol: 8552
['A' 'AA' 'AAAU' ... 'XVZ' 'YGRO' 'YPRO']

Unique values in Url: 12745848
['https://www.nasdaq.com/articles/interesting-a-put-and-call-options-for-august-2024'
 'https://www.nasdaq.com/a

# News dataset refining (handling missing values)

In [3]:
# Drop columns Unnamed: 0 and Url
news_df = news_df.drop(columns=["Unnamed: 0", "Url"])

news_df.head()

,Date,Article_title,Stock_symbol,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,2023-12-16 23:00:00 UTC,Interesting A Put And Call Options For August ...,A,NaN,NaN,"Investors in Agilent Technologies, Inc. (Symbo...",Because the $125.00 strike represents an appro...,The current analytical data (including greeks ...,Below is a chart showing the trailing twelve m...,"At Stock Options Channel, our YieldBoost formu..."
1,2023-12-12 00:00:00 UTC,Wolfe Research Initiates Coverage of Agilent T...,A,NaN,NaN,"Fintel reports that on December 13, 2023, Wolf...","Fintel reports that on December 13, 2023, Wolf...","T. Rowe Price Investment Management holds 10,1...",Agilent Technologies Declares $0.24 Dividend O...,The projected annual revenue for Agilent Techn...
2,2023-12-12 00:00:00 UTC,Agilent Technologies Reaches Analyst Target Price,A,NaN,NaN,"In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...",When a stock reaches the target an analyst has...,When a stock reaches the target an analyst has...
3,2023-12-07 00:00:00 UTC,Agilent (A) Enhances BioTek Cytation C10 With ...,A,NaN,NaN,Agilent Technologies A is enhancing its BioTek...,"Per a Grand View Research report, the global m...","Notably, Agilent enhanced the BioTek Cytation ...","Agilent Technologies, Inc. Price and Consensus...","Notably, Agilent enhanced the BioTek Cytation ..."
4,2023-12-07 00:00:00 UTC,"Pre-Market Most Active for Dec 7, 2023 : SQQQ,...",A,NaN,NaN,The NASDAQ 100 Pre-Market Indicator is up 70.2...,ProShares UltraPro Short QQQ (SQQQ) is -0.15 a...,"As reported by Zacks, the current mean recomme...","The total Pre-Market volume is currently 39,23...",The NASDAQ 100 Pre-Market Indicator is up 70.2...


In [4]:
print(len(news_df))

# Drop rows with missing values in date and stock symbol columns
news_df = news_df.dropna(subset=["Date", "Stock_symbol"])

print(len(news_df))

15549299
5744672


In [5]:
print(len(news_df))

# Drop rows only if they have missing values in each of these columns: Article_title, Publisher, Author, Article, Lsa_summary, Luhn_summary, Textrank_summary, Lexrank_summary
news_df = news_df.dropna(subset=["Article_title", "Publisher", "Author", "Article", "Lsa_summary", "Luhn_summary", "Textrank_summary", "Lexrank_summary"], how="all")

print(len(news_df))

5744672
5744672


In [6]:
# Fill all missing values of the Article_title, Publisher, Author, Article, Lsa_summary, Luhn_summary, Textrank_summary, Lexrank_summary columns with "Unknown"
news_df[["Article_title", "Publisher", "Author", "Article", "Lsa_summary", "Luhn_summary", "Textrank_summary", "Lexrank_summary"]] = news_df[["Article_title", "Publisher", "Author", "Article", "Lsa_summary", "Luhn_summary", "Textrank_summary", "Lexrank_summary"]].fillna("Unknown")

In [7]:
# Check if there are any missing values in the DataFrame
missing_values = news_df.isnull().sum()
print("Missing values in each column:")
print(missing_values[missing_values > 0])

Missing values in each column:
Series([], dtype: int64)


In [8]:
news_df.head()

,Date,Article_title,Stock_symbol,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,2023-12-16 23:00:00 UTC,Interesting A Put And Call Options For August ...,A,Unknown,Unknown,"Investors in Agilent Technologies, Inc. (Symbo...",Because the $125.00 strike represents an appro...,The current analytical data (including greeks ...,Below is a chart showing the trailing twelve m...,"At Stock Options Channel, our YieldBoost formu..."
1,2023-12-12 00:00:00 UTC,Wolfe Research Initiates Coverage of Agilent T...,A,Unknown,Unknown,"Fintel reports that on December 13, 2023, Wolf...","Fintel reports that on December 13, 2023, Wolf...","T. Rowe Price Investment Management holds 10,1...",Agilent Technologies Declares $0.24 Dividend O...,The projected annual revenue for Agilent Techn...
2,2023-12-12 00:00:00 UTC,Agilent Technologies Reaches Analyst Target Price,A,Unknown,Unknown,"In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...",When a stock reaches the target an analyst has...,When a stock reaches the target an analyst has...
3,2023-12-07 00:00:00 UTC,Agilent (A) Enhances BioTek Cytation C10 With ...,A,Unknown,Unknown,Agilent Technologies A is enhancing its BioTek...,"Per a Grand View Research report, the global m...","Notably, Agilent enhanced the BioTek Cytation ...","Agilent Technologies, Inc. Price and Consensus...","Notably, Agilent enhanced the BioTek Cytation ..."
4,2023-12-07 00:00:00 UTC,"Pre-Market Most Active for Dec 7, 2023 : SQQQ,...",A,Unknown,Unknown,The NASDAQ 100 Pre-Market Indicator is up 70.2...,ProShares UltraPro Short QQQ (SQQQ) is -0.15 a...,"As reported by Zacks, the current mean recomme...","The total Pre-Market volume is currently 39,23...",The NASDAQ 100 Pre-Market Indicator is up 70.2...


In [9]:
# Cut the date column to only keep the date part keep only the first 10 characters
news_df["Date"] = news_df["Date"].str[:10]

In [10]:
news_df.head()

,Date,Article_title,Stock_symbol,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,2023-12-16,Interesting A Put And Call Options For August ...,A,Unknown,Unknown,"Investors in Agilent Technologies, Inc. (Symbo...",Because the $125.00 strike represents an appro...,The current analytical data (including greeks ...,Below is a chart showing the trailing twelve m...,"At Stock Options Channel, our YieldBoost formu..."
1,2023-12-12,Wolfe Research Initiates Coverage of Agilent T...,A,Unknown,Unknown,"Fintel reports that on December 13, 2023, Wolf...","Fintel reports that on December 13, 2023, Wolf...","T. Rowe Price Investment Management holds 10,1...",Agilent Technologies Declares $0.24 Dividend O...,The projected annual revenue for Agilent Techn...
2,2023-12-12,Agilent Technologies Reaches Analyst Target Price,A,Unknown,Unknown,"In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...","In recent trading, shares of Agilent Technolog...",When a stock reaches the target an analyst has...,When a stock reaches the target an analyst has...
3,2023-12-07,Agilent (A) Enhances BioTek Cytation C10 With ...,A,Unknown,Unknown,Agilent Technologies A is enhancing its BioTek...,"Per a Grand View Research report, the global m...","Notably, Agilent enhanced the BioTek Cytation ...","Agilent Technologies, Inc. Price and Consensus...","Notably, Agilent enhanced the BioTek Cytation ..."
4,2023-12-07,"Pre-Market Most Active for Dec 7, 2023 : SQQQ,...",A,Unknown,Unknown,The NASDAQ 100 Pre-Market Indicator is up 70.2...,ProShares UltraPro Short QQQ (SQQQ) is -0.15 a...,"As reported by Zacks, the current mean recomme...","The total Pre-Market volume is currently 39,23...",The NASDAQ 100 Pre-Market Indicator is up 70.2...


In [11]:
from tqdm import tqdm
import pandas as pd

# Columns to include in Input_text
cols_to_use = [col for col in news_df.columns if col not in ["Date", "Stock_symbol"]]

def create_input_text_fast(row):
    return "\n".join(f"{col}: {getattr(row, col)}" for col in cols_to_use)

# Create Input_text using itertuples for better performance and memory usage
input_texts = []
for row in tqdm(news_df.itertuples(index=False), total=len(news_df)):
    input_texts.append(create_input_text_fast(row))

100%|██████████| 5744672/5744672 [04:41<00:00, 20392.74it/s] 


In [12]:
# Assign the new column
news_df["Input_text"] = input_texts

# Drop unused columns
news_df = news_df.drop(columns=[
    "Article_title", "Publisher", "Author", "Article", 
    "Lsa_summary", "Luhn_summary", "Textrank_summary", "Lexrank_summary"
])

news_df.head()

,Date,Stock_symbol,Input_text
0,2023-12-16,A,Article_title: Interesting A Put And Call Opti...
1,2023-12-12,A,Article_title: Wolfe Research Initiates Covera...
2,2023-12-12,A,Article_title: Agilent Technologies Reaches An...
3,2023-12-07,A,Article_title: Agilent (A) Enhances BioTek Cyt...
4,2023-12-07,A,Article_title: Pre-Market Most Active for Dec ...


In [13]:
# Save the DataFrame to a CSV file
news_df.to_csv("../data/news/processed.csv", index=False)

# Processed news loading

In [2]:
import os

# Load list of stock symbols from the file
stock_symbols = [f[:-4] for f in os.listdir("../data/normalized/sp500/csv") if f.endswith(".csv")]
# Remove the .DS_Store file if it exists
if ".DS_Store" in stock_symbols:
    stock_symbols.remove(".DS_Store")

print(stock_symbols)
print(len(stock_symbols))

['A', 'AAL', 'AAP', 'AAPL', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'AMAT', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'AON', 'AOS', 'APA', 'APD', 'APH', 'ARE', 'ATVI', 'AVB', 'AVY', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF-A', 'BIIB', 'BIO', 'BK', 'BLK', 'BMRA', 'BMY', 'BR', 'BRK-A', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CCI', 'CDE', 'CDNS', 'CF', 'CHD', 'CHRW', 'CINF', 'CL', 'CLX', 'CME', 'CMG', 'CMI', 'CNC', 'CNP', 'COO', 'COP', 'COST', 'COWN', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CTAS', 'CTSH', 'CUK', 'D', 'DAL', 'DE', 'DFS', 'DGX', 'DHI', 'DIS', 'DLTR', 'DOV', 'DPZ', 'DRI', 'DTE', 'DVA', 'DXCM', 'EA', 'EBAY', 'ECL', 'ED', 'EFX', 'EIX', 'EL', 'EMN', 'EMR', 'ENS', 'EOG', 'EQIX', 'EQR', 'ES', 'ESS', 'EW', 'EXR', 'F', 'FAST', 'FCX', 'FDX', 'FE', 'FFIV', 'FIS', 'FISV', 'FITB', 'FLS', 'FMC', 'FRT', 'FTI', 'GD', 'GE', 'GGG', 'GILD', 'GIS', 'GOOG', 'GPC', 'GPN', 'GRMN', 'GWW', 'HAL', '

In [3]:
path = "../data/news/processed.csv"

news_df = pd.read_csv(path)

In [4]:
# Remove rows where Stock_symbol is not in the list of stock symbols
print(len(news_df))
news_df = news_df[news_df["Stock_symbol"].isin(stock_symbols)]
print(len(news_df))

5744672
1016283


In [6]:
# Iterate over the stock symbols and create a separate CSV file for each
for stock_symbol in stock_symbols:
    print("Processing stock symbol:", stock_symbol)
    # Filter the DataFrame for the current stock symbol
    filtered_df = news_df[news_df["Stock_symbol"] == stock_symbol]
    
    # Save the filtered DataFrame to a CSV file
    filtered_df.to_csv(f"../data/news/single_stocks/{stock_symbol}.csv", index=False)

Processing stock symbol: A
Processing stock symbol: AAL
Processing stock symbol: AAP
Processing stock symbol: AAPL
Processing stock symbol: ABC
Processing stock symbol: ABMD
Processing stock symbol: ABT
Processing stock symbol: ACN
Processing stock symbol: ADI
Processing stock symbol: ADM
Processing stock symbol: ADP
Processing stock symbol: ADSK
Processing stock symbol: AEE
Processing stock symbol: AEP
Processing stock symbol: AIZ
Processing stock symbol: AJG
Processing stock symbol: AKAM
Processing stock symbol: ALB
Processing stock symbol: ALGN
Processing stock symbol: ALK
Processing stock symbol: AMAT
Processing stock symbol: AMD
Processing stock symbol: AME
Processing stock symbol: AMGN
Processing stock symbol: AMP
Processing stock symbol: AMT
Processing stock symbol: AMZN
Processing stock symbol: AON
Processing stock symbol: AOS
Processing stock symbol: APA
Processing stock symbol: APD
Processing stock symbol: APH
Processing stock symbol: ARE
Processing stock symbol: ATVI
Process

# FinBert model test

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [38]:
# Take first row to test
row = news_df.iloc[0]

# Create the input text by concatenating the columns with their names and values
input_text = ""
for col in news_df.columns:
    # Skip the columns that are not needed for the input text
    if not col in ["Date", "Stock_symbol"]:
        input_text += f"{col}: {row[col]}\n"
print(input_text)

Article_title: Interesting A Put And Call Options For August 2024
Publisher: Unknown
Author: Unknown
Article: Investors in Agilent Technologies, Inc. (Symbol: A) saw new options begin trading this week, for the August 2024 expiration. One of the key inputs that goes into the price an option buyer is willing to pay, is the time value, so with 241 days until expiration the newly trading contracts represent a possible opportunity for sellers of puts or calls to achieve a higher premium than would be available for the contracts with a closer expiration. At Stock Options Channel, our YieldBoost formula has looked up and down the A options chain for the new August 2024 contracts and identified one put and one call contract of particular interest.
The put contract at the $125.00 strike price has a current bid of $4.50. If an investor was to sell-to-open that put contract, they are committing to purchase the stock at $125.00, but will also collect the premium, putting the cost basis of the sha

In [39]:
input_tokenized = tokenizer(input_text, return_tensors="pt")
print(input_tokenized)
print(len(input_tokenized["input_ids"][0]))

{'input_ids': tensor([[ 101, 3720, 1035,  ..., 5067, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
1711


In [48]:
from math import ceil
import torch

sentiments = []

for i in range(ceil(len(input_tokenized["input_ids"][0]) / 512)):
    # Get the start and end indices for the current chunk
    start = i * 512
    end = min((i + 1) * 512, len(input_tokenized["input_ids"][0]))

    # Get the input IDs for the current chunk
    input_ids_chunk = input_tokenized["input_ids"][0][start:end].unsqueeze(0)

    # Get the attention mask for the current chunk
    attention_mask_chunk = input_tokenized["attention_mask"][0][start:end].unsqueeze(0)

    # Get the model output for the current chunk
    with torch.no_grad():
        output = model(input_ids_chunk, attention_mask=attention_mask_chunk)
        logits = output.logits

    # Get the predicted sentiment for the current chunk
    sentiment = logits[0]
    # Append sentiment value and the length of the chunk that will be used as a weight for the average to the sentiments list
    sentiments.append((sentiment, end - start))

In [49]:
print(sentiments)

[(tensor([-0.4522, -1.1169,  2.0375]), 512), (tensor([-0.5175, -1.2336,  2.1715]), 512), (tensor([-0.3820, -1.3655,  1.9722]), 512), (tensor([-0.7449, -1.1740,  2.2532]), 175)]


In [50]:
# Average the sentiments using the weights
weighted_sentiment = sum([sentiment[0] * sentiment[1] for sentiment in sentiments]) / sum([sentiment[1] for sentiment in sentiments])
print(weighted_sentiment)

tensor([-0.4807, -1.2320,  2.0801])


In [ ]:
test = [
    torch.tensor([0.1, 0.2, 0.3]),
    torch.tensor([0.4, 0.5, 0.6]),
    torch.tensor([0.7, 0.8, 0.9])
]

# Average the tensors
average_tensor = sum(test) / len(test)
print(average_tensor[0])

0.4000000059604645


# Sentiment processing

In [115]:
import os
import numpy as np

def compute_sentiment_for_stock(stock_df, stock_symbol, date):
    news_for_stock = get_news_for_stock(stock_symbol, date)

    # Add the number of news found to the stock DataFrame
    stock_df.at[index, "News_count"] = len(news_for_stock)

    # If there are no news articles for the stock on that date, set the sentiment to 0
    sentiments = [0] * 3

    print(f"Computing sentiment for {len(news_for_stock)} news articles")
    if len(news_for_stock) > 0:
        sentiments = []
        for _, row in news_for_stock.iterrows():
            sentiment = compute_sentiment(row)
            sentiments.append(sentiment)

        # Average the sentiments
        sentiments = sum(sentiments) / len(sentiments)
        sentiments = [np.float64(tensor.item()) for tensor in sentiments]
    
    print("Adding sentiment to stock DataFrame")
    # Add the average sentiment to the stock DataFrame
    for i in range(len(sentiments)):
        stock_df.at[index, f"Sentiment_{i}"] = sentiments[i]


def compute_sentiment(row):
    input_tokenized = tokenize_input(row)
    sentiments = []

    for i in range(ceil(len(input_tokenized["input_ids"][0]) / 512)):
        # Get the start and end indices for the current chunk
        start = i * 512
        end = min((i + 1) * 512, len(input_tokenized["input_ids"][0]))

        # Get the input IDs for the current chunk
        input_ids_chunk = input_tokenized["input_ids"][0][start:end].unsqueeze(0)

        # Get the attention mask for the current chunk
        attention_mask_chunk = input_tokenized["attention_mask"][0][start:end].unsqueeze(0)

        # Get the model output for the current chunk
        with torch.no_grad():
            output = model(input_ids_chunk, attention_mask=attention_mask_chunk)
            logits = output.logits

        # Get the predicted sentiment for the current chunk
        sentiment = logits[0]
        # Append sentiment value and the length of the chunk that will be used as a weight for the average to the sentiments list
        sentiments.append((sentiment, end - start))

    # Average the sentiments using the weights
    weighted_sentiment = sum([sentiment[0] * sentiment[1] for sentiment in sentiments]) / sum([sentiment[1] for sentiment in sentiments])
    return weighted_sentiment

def tokenize_input(row):
    # Create the input text by concatenating the columns with their names and values
    input_text = ""
    for col in news_df.columns:
        # Skip the columns that are not needed for the input text
        if not col in ["Date", "Stock_symbol"]:
            input_text += f"{col}: {row[col]}\n"

    return tokenizer(input_text, return_tensors="pt")

def get_stock_files():
    # Get the list of all stock csv files in the directory
    stock_files = ["../data/normalized/sp500/csv/" + f for f in os.listdir("../data/normalized/sp500/csv") if f.endswith(".csv")]
    # Remove the .DS_Store file if it exists
    if ".DS_Store" in stock_files:
        stock_files.remove(".DS_Store")
    return stock_files

def get_news_for_stock(stock_symbol, date):
    # Filter the news DataFrame to get the rows that match the stock symbol and date
    news_for_stock = news_df[(news_df["Stock_symbol"] == stock_symbol) & (news_df["Date"] == date)]

    return news_for_stock

* For each stock csv file
    * For each entry
        * Find the news from news_df with the same date and stock symbol
        * Compute the sentiment of all the news
        * Average the sentiment of all the news
        * Add to the stock csv files 3 columns (1 for each sentiment component)
        * Add to the stock csv files a column that contains the number of news found for each row

In [96]:
stock_files = get_stock_files()
print(stock_files)

['../data/normalized/sp500/csv/A.csv', '../data/normalized/sp500/csv/AAL.csv', '../data/normalized/sp500/csv/AAP.csv', '../data/normalized/sp500/csv/AAPL.csv', '../data/normalized/sp500/csv/ABC.csv', '../data/normalized/sp500/csv/ABMD.csv', '../data/normalized/sp500/csv/ABT.csv', '../data/normalized/sp500/csv/ACN.csv', '../data/normalized/sp500/csv/ADI.csv', '../data/normalized/sp500/csv/ADM.csv', '../data/normalized/sp500/csv/ADP.csv', '../data/normalized/sp500/csv/ADSK.csv', '../data/normalized/sp500/csv/AEE.csv', '../data/normalized/sp500/csv/AEP.csv', '../data/normalized/sp500/csv/AIZ.csv', '../data/normalized/sp500/csv/AJG.csv', '../data/normalized/sp500/csv/AKAM.csv', '../data/normalized/sp500/csv/ALB.csv', '../data/normalized/sp500/csv/ALGN.csv', '../data/normalized/sp500/csv/ALK.csv', '../data/normalized/sp500/csv/AMAT.csv', '../data/normalized/sp500/csv/AMD.csv', '../data/normalized/sp500/csv/AME.csv', '../data/normalized/sp500/csv/AMGN.csv', '../data/normalized/sp500/csv/AMP.

In [ ]:
import numpy as np

stock_files = get_stock_files()
stock_files = stock_files[:1]

for stock_file in stock_files:
    print(f"Processing file: {stock_file}")

    # Read the stock file
    stock_df = pd.read_csv(stock_file)

    stock_symbol = stock_df["Stock_symbol"].iloc[0]

    for index, row in stock_df.iterrows():
        date = row["Date"]

        compute_sentiment_for_stock(stock_df, stock_symbol, date)
        
    stock_df.head()
        

Processing file: ../data/normalized/sp500/csv/A.csv
Getting news for date: 2008-01-02
Computing sentiment for 0 news articles
Adding sentiment to stock DataFrame
Getting news for date: 2008-01-03
Computing sentiment for 0 news articles
Adding sentiment to stock DataFrame
Getting news for date: 2008-01-04
Computing sentiment for 0 news articles
Adding sentiment to stock DataFrame
Getting news for date: 2008-01-07
Computing sentiment for 0 news articles
Adding sentiment to stock DataFrame
Getting news for date: 2008-01-08
Computing sentiment for 0 news articles
Adding sentiment to stock DataFrame
Getting news for date: 2008-01-09
Computing sentiment for 0 news articles
Adding sentiment to stock DataFrame
Getting news for date: 2008-01-10
Computing sentiment for 0 news articles
Adding sentiment to stock DataFrame
Getting news for date: 2008-01-11
Computing sentiment for 0 news articles
Adding sentiment to stock DataFrame
Getting news for date: 2008-01-14
Computing sentiment for 0 news art

AttributeError: 'torch.dtype' object has no attribute 'kind'

In [103]:
stock_df[stock_df["News_count"] > 0].head()

,Date,Stock_symbol,Low,Open,Volume,High,Close,Adjusted Close,ABER_ZG_5_15,ABER_SG_5_15,...,VWMA_10,WCP,WILLR_14,WMA_10,ZL_EMA_10,ZS_30,News_count,Sentiment_0,Sentiment_1,Sentiment_2
333,2009-04-29,A,-1.130248,-1.131785,1.021779,-1.117936,-1.117422,-1.083444,-1.137467,-1.131569,...,-1.138975,-1.12078,1.163775,-1.136809,-1.129139,1.460558,1.0,NaN,NaN,NaN


In [107]:
type(stock_df["Low"][0])

numpy.float64

In [112]:
news_for_stock = get_news_for_stock("A", "2009-04-29")

sentiments = [0] * 3
if len(news_for_stock) > 0:
    sentiments = []
    for _, row in news_for_stock.iterrows():
        sentiment = compute_sentiment(row)
        sentiments.append(sentiment)

    # Average the sentiments
    sentiments = sum(sentiments) / len(sentiments)

import numpy as np
# convert tensor to numpy float64
sentiments = [np.float64(tensor.item()) for tensor in sentiments]
print(sentiments)
print(type(sentiments[0]))

[-1.5017825365066528, 0.2473420947790146, 2.3462471961975098]
<class 'numpy.float64'>


# TODO

* Check FinBert input size
* Decide how to pass the news info: (GCC mi dai un parere su questo?)
    * Only the title as in the paper?
    * Add also Author and summaries values?
    * How to add them? Simple concatenation?
    * Maybe doing sentiment on each independently and them averaging the sentiment results?
* For each stock retrieve relative news for each date, compute sentiment and add score to the stock csv file